# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,41.76,97,100,30.82,TF,1720634516
1,1,santa ana,33.7456,-117.8678,88.32,68,40,9.22,US,1720634316
2,2,georgetown,5.4112,100.3354,82.35,69,20,6.91,MY,1720634320
3,3,waitangi,-43.9535,-176.5597,46.81,91,100,11.99,NZ,1720634370
4,4,ribeira grande,38.5167,-28.7000,79.20,57,20,14.00,PT,1720634522


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    alpha=0.5

)

# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 70) &
    (city_data_df['Max Temp'] < 95) &
    (city_data_df['Wind Speed'] < 3.5) &
    (city_data_df['Cloudiness'] < 10)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
98,98,vernon,50.2581,-119.2691,93.61,32,5,2.30,CA,1720634460
113,113,remire-montjoly,4.9167,-52.2667,80.64,83,3,3.44,GF,1720634686
142,142,coursan,43.2337,3.0571,82.92,45,0,1.99,FR,1720634731
156,156,carauari,-4.8828,-66.8958,93.29,41,6,2.39,BR,1720634751
224,224,portland,45.5234,-122.6762,86.61,35,0,1.99,US,1720634403
256,256,arenapolis,-14.4503,-56.8461,85.50,30,1,2.91,BR,1720634902
270,270,fort mcmurray,56.7268,-111.3810,85.59,48,0,3.44,CA,1720634736
275,275,santo antonio do ica,-3.1022,-67.9397,89.98,51,8,3.24,BR,1720634931
371,371,williams lake,52.1415,-122.1445,87.15,26,4,1.95,CA,1720634905
398,398,presidente medici,-11.1753,-61.9014,94.60,29,0,1.54,BR,1720635128


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
98,vernon,CA,50.2581,-119.2691,32,
113,remire-montjoly,GF,4.9167,-52.2667,83,
142,coursan,FR,43.2337,3.0571,45,
156,carauari,BR,-4.8828,-66.8958,41,
224,portland,US,45.5234,-122.6762,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    #'limit': 1,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vernon - nearest hotel: Okanagan Royal Park Inn
remire-montjoly - nearest hotel: Complexe Belova
coursan - nearest hotel: Hôtel le Mas Pierrot
carauari - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
arenapolis - nearest hotel: No hotel found
fort mcmurray - nearest hotel: Nomad Hotel and Suites
santo antonio do ica - nearest hotel: No hotel found
williams lake - nearest hotel: Coast Fraser Inn
presidente medici - nearest hotel: No hotel found
hanover - nearest hotel: Am Leineschloss
hilo - nearest hotel: Dolphin Bay Hotel
tarauaca - nearest hotel: Hotel Estrela
de queen - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
98,vernon,CA,50.2581,-119.2691,32,Okanagan Royal Park Inn
113,remire-montjoly,GF,4.9167,-52.2667,83,Complexe Belova
142,coursan,FR,43.2337,3.0571,45,Hôtel le Mas Pierrot
156,carauari,BR,-4.8828,-66.8958,41,No hotel found
224,portland,US,45.5234,-122.6762,35,the Hoxton
256,arenapolis,BR,-14.4503,-56.8461,30,No hotel found
270,fort mcmurray,CA,56.7268,-111.3810,48,Nomad Hotel and Suites
275,santo antonio do ica,BR,-3.1022,-67.9397,51,No hotel found
371,williams lake,CA,52.1415,-122.1445,26,Coast Fraser Inn
398,presidente medici,BR,-11.1753,-61.9014,29,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [40]:
%%capture --no-display

# Configure the map plot
map_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    color="City",
    alpha=0.5,
    hover_cols=["Hotel Name", "Country"]

)

# Display the map
map_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)